In [18]:
https://apps.dtic.mil/dtic/tr/fulltext/u2/a548850.pdf

In [18]:
import re
from subprocess import call, Popen, PIPE

In [18]:
class ShadowCopy:
    def __init__(self,attrs):
        self.attrs=attrs
    def originatingMachine(self):
        return self.attrs['Originating Machine']
    def ctime(self):
        return self.attrs['creation time']
    def volumePath(self):
        return self.attrs['Shadow Copy Volume']
    def volumeName(self):
        return self.volumePath().split('\\')[-1]
    

In [18]:
def vssadminlistparse(vssadminout):
    ret=[]
    current=None
    fix1 = re.compile("Contained.∗copies at ")
    for line in vssadminout.splitlines():
        if line=="":
            if current:
                ret.append(ShadowCopy(current))
                current=None
            continue
        if line.startswith("Contents of shadow copy set ID: "):
            current={}#startofthenewone
            id=line.split(":")[1][1:]
            current['id']=id
            continue
        if not current:
            continue
       
        line=line.strip()
        colon=line.find(':')
        if colon>=0:
            name=line[0:colon]
            value=line[colon+2:]
            if name.endswith("creation time"): name="creation time"
            current[name]=value
    return ret

In [18]:
def availableVolumes():
    listoutput=Popen(['vssadmin.exe','list','shadows'],stdout=PIPE).communicate()[0]
    listoutput=listoutput.decode('utf−8')
    return vssadminlistparse(listoutput)

In [35]:
import os
def shadowFile(path, drive):
    drive_letter = path[0]
    new_path = path.replace(drive_letter + ':',drive,1)
    if new_path == path:
        raise Exception("Problem processing path: {0}".format(path))
    if os.path.exists(new_path):
        return new_path
    return None

In [72]:
def findShadowFile(path):
    for item in availableVolumes():
        sf = shadowFile(path,item.volumePath())
        print(
            item.volumeName(),
            item.ctime(),
            item.originatingMachine(),
            item.volumePath(),
            sf
        )
        if sf is not None:
            with open(sf,'rb') as f:
                print(len(f.read()))
        print()

In [73]:
findShadowFile("C:\\Windows\\System32\\config\\SAM")

In [64]:
findShadowFile("C:\\Windows\\System32\\config\\SYSTEM")